# FI-2Pop demo

Demo notebook on the integration of FI-2Pop with the L-System.

## Imports

In [1]:
import json

GECCO-compatible `matplotlib` options:

In [2]:
import matplotlib

matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

Import `PCGSEPy` modules:

In [3]:
from pcgsepy.common.vecs import orientation_from_str, Vec 
from pcgsepy.config import COMMON_ATOMS, HL_ATOMS, N_ITERATIONS, REQ_TILES 
from pcgsepy.lsystem.rules import RuleMaker
from pcgsepy.lsystem.actions import AtomAction, Rotations
from pcgsepy.lsystem.parser import HLParser, LLParser
from pcgsepy.lsystem.solver import LSolver
from pcgsepy.lsystem.constraints import ConstraintHandler, ConstraintLevel, ConstraintTime
from pcgsepy.lsystem.constraints_funcs import components_constraint, intersection_constraint, symmetry_constraint, wheels_plane_constraint
from pcgsepy.lsystem.lsystem import LSystem
from pcgsepy.structure import block_definitions

## Setup

In [4]:
import logging

logging.basicConfig()
logger = logging.getLogger('base-logger')
logger.setLevel(logging.WARNING)

In [5]:
with open(COMMON_ATOMS, "r") as f:
    common_alphabet = json.load(f)

for k in common_alphabet:
    action, args = common_alphabet[k]["action"], common_alphabet[k]["args"]
    action = AtomAction(action)
    if action == AtomAction.MOVE:
        args = orientation_from_str[args]
    elif action == AtomAction.ROTATE:
        args = Rotations(args)
    common_alphabet[k] = {"action": action, "args": args}

In [6]:
with open(HL_ATOMS, "r") as f:
    hl_atoms = json.load(f)

tiles_dimensions = {}
tiles_block_offset = {}
for tile in hl_atoms.keys():
    dx, dy, dz = hl_atoms[tile]["dimensions"]
    tiles_dimensions[tile] = Vec.v3i(dx, dy, dz)
    tiles_block_offset[tile] = hl_atoms[tile]["offset"]

hl_alphabet = {}
for k in common_alphabet.keys():
    hl_alphabet[k] = common_alphabet[k]

for hk in hl_atoms.keys():
    hl_alphabet[hk] = {"action": AtomAction.PLACE, "args": []}

In [7]:
ll_alphabet = {}

for k in common_alphabet.keys():
    ll_alphabet[k] = common_alphabet[k]

for k in block_definitions.keys():
    if k != "":  # TODO: This is a probable bug, reported to the SE API devs
        ll_alphabet[k] = {"action": AtomAction.PLACE, "args": [k]}

## L-System components

In [8]:
hl_rules = RuleMaker(ruleset='hlrules').get_rules()
ll_rules = RuleMaker(ruleset='llrules').get_rules()

hl_parser = HLParser(rules=hl_rules)
ll_parser = LLParser(rules=ll_rules)

hl_solver = LSolver(parser=hl_parser,
                    atoms_alphabet=hl_alphabet,
                    extra_args={
                        'tiles_dimensions': tiles_dimensions,
                        'tiles_block_offset': tiles_block_offset,
                        'll_rules': ll_rules
                    })
ll_solver = LSolver(parser=ll_parser,
                    atoms_alphabet=dict(hl_alphabet, **ll_alphabet),
                    extra_args={})

In [9]:
rcc = ConstraintHandler(
    name="required_components",
    level=ConstraintLevel.HARD_CONSTRAINT,
    when=ConstraintTime.END,
    f=components_constraint,
    extra_args={
        'alphabet': hl_alphabet
    }
)
rcc.extra_args["req_tiles"] = REQ_TILES

nic = ConstraintHandler(
    name="no_intersections",
    level=ConstraintLevel.HARD_CONSTRAINT,
    when=ConstraintTime.DURING,
    f=intersection_constraint,
    extra_args={
        'alphabet': dict(hl_alphabet, **ll_alphabet)
    },
    needs_ll=True
)
nic.extra_args["tiles_dimensions"] = tiles_dimensions

sc = ConstraintHandler(
    name="symmetry",
    level=ConstraintLevel.SOFT_CONSTRAINT,
    when=ConstraintTime.END,
    f=symmetry_constraint,
    extra_args={
        'alphabet': dict(hl_alphabet, **ll_alphabet)
    }
)

wopc = ConstraintHandler(
    name="wheels_on_plane",
    level=ConstraintLevel.SOFT_CONSTRAINT,
    when=ConstraintTime.END,
    f=wheels_plane_constraint,
    extra_args={
        'alphabet': dict(hl_alphabet, **ll_alphabet)
    }
)

In [10]:
lsystem = LSystem(
    hl_solver=hl_solver, ll_solver=ll_solver
)

In [11]:
lsystem.add_hl_constraint(c=rcc)
lsystem.add_hl_constraint(c=nic)

lsystem.add_ll_constraint(c=sc)
lsystem.add_ll_constraint(c=wopc)

## FI-2Pop

In [37]:
feasible_pop = []
infeasible_pop = []

feasible_pool = []
infeasible_pool = []

In [38]:
pops_size = 50
n_retries = 100


i = 0
while len(feasible_pop) < pops_size or len(infeasible_pop) < pops_size:
    lsystem.check_sat = False
    hl_axioms = lsystem.get_hl_axioms(starting_axiom="begin",
                                      iterations=N_ITERATIONS)

    axioms_sats = {}
    for hl_axiom in hl_axioms:
        axioms_sats[hl_axiom] = {}
        for t in [ConstraintTime.DURING, ConstraintTime.END]:
            sat = lsystem.hlsolver._check_constraints(axiom=hl_axiom,
                                                      when=t)
            axioms_sats[hl_axiom][t] = sat

    to_expand_further = []
    for hl_axiom in axioms_sats.keys():
        if axioms_sats[hl_axiom][ConstraintTime.DURING][ConstraintLevel.HARD_CONSTRAINT] and axioms_sats[hl_axiom][ConstraintTime.END][ConstraintLevel.HARD_CONSTRAINT]:
            to_expand_further.append(hl_axiom)
        else:
            if hl_axiom not in infeasible_pop and len(infeasible_pop) < pops_size:
                infeasible_pop.append(hl_axiom)
    ml_axioms = lsystem.get_ml_axioms(hl_axioms=to_expand_further)
    lsystem.check_sat = True
    ll_axioms, to_rem = lsystem.get_ll_axioms(ml_axioms)

    for r in to_rem:
        infeasible_pop.append(to_expand_further[r])
    for n, hl_axiom in enumerate(to_expand_further):
        if n not in to_rem and len(feasible_pop) < pops_size:
            feasible_pop.append(hl_axiom)

    i += 1
    if i == n_retries:
        break

In [39]:
feasible_pop

['cockpitcorridor(9)corridor(9)[RotZccwXcorridor(6)wheels][RotZccwYcorridor(5)corridor(5)thruster]',
 'cockpitcorridor(2)[RotZccwXcorridor(5)][RotZccwYcorridor(6)thruster]corridor(7)corridor(7)[RotZccwYcorridor(3)thruster]',
 'cockpitcorridor(2)[RotZccwXcorridor(5)][RotZccwYcorridor(6)wheels]corridor(3)corridor(3)[RotZccwYcorridor(3)thruster]',
 'cockpitcorridor(5)[RotZccwXcorridor(9)]corridor(5)[RotZccwXcorridor(5)][RotZccwYcorridor(6)[RotZccwYcorridor(5)]thruster]',
 'cockpitcorridor(4)[RotZccwXcorridor(5)]corridor(4)[RotZccwXcorridor(4)][RotZccwYcorridor(2)[RotZccwXcorridor(4)]thruster]',
 'cockpitcorridor(3)corridor(3)[RotZccwXcorridor(2)thruster][RotZccwYcorridor(8)[RotZccwXcorridor(2)]wheels]',
 'cockpitcorridor(4)corridor(4)[RotZccwXcorridor(2)thruster][RotZccwYcorridor(5)[RotZccwXcorridor(6)]thruster]',
 'cockpitcorridor(7)[RotZccwYcorridor(4)]corridor(4)corridor(4)[RotZccwYcorridor(6)[RotZccwXcorridor(7)]thruster]',
 'cockpitcorridor(3)corridor(3)[RotZccwXcorridor(6)thruster]c

In [40]:
infeasible_pop

['cockpitcorridor(9)corridor(9)[RotZccwYcorridor(1)thruster][RotZccwYcorridor(3)[RotZccwXcorridor(6)]wheels]',
 'cockpitcorridor(7)[RotZccwYcorridor(2)][RotZccwYcorridor(7)wheels][RotZccwXcorridor(1)[RotZccwYcorridor(6)]thruster]',
 'cockpitcorridor(7)[RotZccwYcorridor(6)][RotZccwXcorridor(3)wheels][RotZccwXcorridor(1)[RotZccwXcorridor(1)]thruster]',
 'cockpitcorridor(8)corridor(8)[RotZccwYcorridor(1)thruster][RotZccwYcorridor(3)[RotZccwXcorridor(8)]wheels]',
 'cockpitcorridor(9)[RotZccwYcorridor(6)][RotZccwXcorridor(4)thruster][RotZccwYcorridor(6)[RotZccwXcorridor(3)]wheels]',
 'cockpitcorridor(5)[RotZccwXcorridor(4)]corridor(5)[RotZccwYcorridor(4)][RotZccwYcorridor(2)corridor(2)thruster]',
 'cockpitcorridor(2)[RotZccwXcorridor(6)][RotZccwXcorridor(3)thruster]corridor(2)[RotZccwYcorridor(9)][RotZccwXcorridor(4)wheels]',
 'cockpitcorridor(7)corridor(7)[RotZccwXcorridor(4)wheels][RotZccwYcorridor(6)[RotZccwXcorridor(1)]wheels]',
 'cockpitcorridor(7)[RotZccwYcorridor(6)][RotZccwXcorridor

In [41]:
len(feasible_pop)

50

In [42]:
len(infeasible_pop)

50